# Quelques techniques courantes

## Insertion de contenu textuel

La fonction XSL `text` permet d’insérer facilement tout caractère textuel.

### Insérer du texte brut

```xml
<!-- plain text -->
<xsl:text>Nom :</xsl:text>

<!-- a whitespace -->
<xsl:text> </xsl:text>
```

### Insérer une balise autonome

```xml
<!-- special characters are printed as is -->
<xsl:text disable-output-escaping="yes">&amp;lt;br/&amp;gt;</xsl:text>
```

### Insérer un caractère ASCII

```xml
<!-- hexadécimal notation (backspace) -->
<xsl:text>&amp;#xA;</xsl:text>
<!-- decimal notation (backspace) -->
<xsl:text>&amp;#10;</xsl:text>
```

## Le recours aux variables

### Stocker un motif récurrent

```xml
<!-- delimiter into a variable -->
<xsl:variable name="delimiter" select="','"/>

<!-- call to the variable -->
<xsl:value-of select="data-1"/>
<xsl:value-of select="$delimiter"/>
<xsl:value-of select="data-2"/>
<xsl:value-of select="$delimiter"/>
<xsl:value-of select="data-3"/>
```

### Stocker le résultat d’une expression XPath complexe

```xml
<!-- store into variable named 'carina' -->
<xsl:variable name="carina" select="//constellation[name = 'Carina']"/>

<!-- calls to the variable 'carina' -->
<xsl:value-of select="$carina/name/@fr"/>
<xsl:value-of select="$carina//area[@unit = 'deg2']"/>

<!-- without calling the variable -->
<xsl:value-of select="//constellation[name = 'Carina']name/@fr"/>
<xsl:value-of select="//constellation[name = 'Carina']//area[@unit = 'deg2']"/>
```

**Remarque :** recourir aux variables est plus rapide que d’évaluer des expressions XPath complètes.

```xml
<!-- variables -->
<xsl:variable name="c1" select="//constellation[1]"/>
<xsl:variable name="c2" select="//constellation[2]"/>

<!-- call to variables -->
<table>
    <tr>
        <th><xsl:value-of select="$c1/name"/></th>
        <th><xsl:value-of select="$c2/name"/></th>
    </tr>
    <tr>
        <td><xsl:value-of select="$c1//area[@unit = 'percent']"/></td>
        <td><xsl:value-of select="$c2//area[@unit = 'percent']"/></td>
    </tr>
    <tr>
        <td><xsl:value-of select="$c1//area[@unit = 'deg2']"/></td>
        <td><xsl:value-of select="$c2//area[@unit = 'deg2']"/></td>
    </tr>
</table>
```

## Évaluer des expressions XPath

### Éléments XSL et non-XSL

Le préfixe `xsl` identifie un élément dans l’espace de nommage qui fait référence à la grammaire XSL :

```xml
<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform" version="3.0">
    <!-- transformations -->
</xsl:stylesheet>
```

À partir de ce moment, les éléments préfixés `xsl` ont une fonction bien définie :

```xml
<!-- XSL element 'output' -->
<xsl:output/>

<!-- undefined element 'output' -->
<output/>
```

Les éléments XSL sont prévus pour évaluer des expressions XPath :

```xml
<xsl:value-of select="//constellation[4]/name"/>
```

En revanche, évaluer une expression XPath dans un élément non-XSL se fait en l’encadrant d’accolades :

```xml
<!-- curly braces allow XPath expressions -->
<areas constellation="{//constellation[4]/name}"/>
```

### Court-circuitage avec la fonction `choose`

La structure `choose` est idéale pour éliminer plusieurs conditions simples :

```xml
<xsl:variable name="puc" select="//Equipe[@NomClub = 'Paris']"/>
<xsl:variable name="lyon" select="//Equipe[@NomClub = 'Lyon']"/>
<xsl:variable name="nancy" select="//Equipe[@NomClub = 'Nancy']"/>
<xsl:choose>
    <xsl:when test="$puc/number(Points) > $lyon/number(Points)"/>
    <xsl:when test="$puc/number(Points) > $nancy/number(Points)"/>
    <xsl:otherwise>
        <xsl:call-template name="alerteCoach"/>
    </xsl:otherwise>
</xsl:choose>
```

Elle est préférable à une expression XPath complexe :

```xml
<xsl:if test="(//Equipe[@NomClub = 'Paris']/number(Points) &lt; //Equipe[@NomClub = 'Nancy']/number(Points) and //Equipe[@NomClub = 'Paris']/number(Points) &lt; //Equipe[@NomClub = 'Lyon']/number(Points))">
    <xsl:call-template name="alerteCoach"/>
</xsl:if>
```

## Les structures itératives

La fonction `for-each` permet d’appliquer un traitement identique à chaque élément d’un ensemble de nœuds :

```xml
<xsl:for-each select="//constellation">
    <tr>
        <td><xsl:value-of select="@origin"/></td>
        <td><xsl:value-of select="name"/></td>
    </tr>
</xsl:for-each>
```

Elle accepte également la fonction `sort` pour le tri des données.

```xml
<xsl:for-each select="//constellation">
    <xsl:sort select="areas/area[@unit = 'deg2']" data-type="number" order="descending"/>
    <tr>
        <td><xsl:value-of select="@origin"/></td>
        <td><xsl:value-of select="name"/></td>
    </tr>
</xsl:for-each>
```

## Générer des identifiants uniques

Chaque élément dispose d’un identifiant unique que la fonction XPath `generate-id()` révèle :

```xml
<xsl:for-each select="//constellation">
    <xsl:element name="a">
        <xsl:attribute name="href">
            <xsl:value-of select="generate-id()"/>
        </xsl:attribute>
        <xsl:value-of select="name"/>
    </xsl:element>
</xsl:for-each>
```

Syntaxe équivalente à :

```xml
<xsl:for-each select="//constellation">
    <a href="#{generate-id()}"><xsl:value-of select="name"/></a>
</xsl:for-each>
```

## Regrouper des données

L’idée est de regrouper des données partageant une même clé sous une seule entrée. Par exemple, dans le fichier sur les [constellations](./files/xml/constellations.xml), plusieurs d’entre elles ont été découvertes par Ptolémée et d’autres par Lacaille.

L’objectif ici est de créer un nouveau fichier XML en regroupant les constellations par le nom de leur découvreur, dans une liste triée alphabétiquement :

```xml
<astronoms>
    <astronom name="Lacaille">
        <constellation>Carina</constellation>
        <constellation>Microscopium</constellation>
        …
    </astronom>
    <astronom name="Ptolémée">
        <constellation>Aquila</constellation>
        <constellation>Auriga</constellation>
        …
    </astronom>
</astronoms>
```

### La méthode muenchienne

Méthode ficelée par Steve Muench pour filtrer les doublons. Elle repose sur les fonctions XSL `key` et XPath `generate-id()`.

La première étape consiste à créer un index des entrées à dédoublonner avec la fonction `key`, à déclarer comme enfant de l’élément racine `stylesheet` :

```xml
<!-- astronoms -->
<xsl:key name="astronoms" match="constellation" use="@origin"/>
```

Ensuite, une fois la clé définie, la fonction `key` permet de sélectionner selon un paramètre :

```xml
<!-- each time the astronom is 'Lacaille' -->
<xsl:for-each select="key('astronoms', 'Lacaille')">
    <!-- LacailleLacailleLacaille -->
    <astronom><xsl:value-of select="@origin"/></astronom>
</xsl:for-each>
```

Les données étant redondantes, il suffit de sélectionner la première occurrence :

```xml
<xsl:for-each select="key('astronoms', 'Lacaille')[1]">
    <!-- Lacaille -->
    <astronom><xsl:value-of select="@origin"/></astronom>
</xsl:for-each>
```

La dernière étape consiste à envoyer dynamiquement le nom de tous les découvreurs, nom qui est disponible dans l’attribut `@origin` :

```xml
<xsl:for-each select="//constellation[key('astronoms', @origin)[1]]">
    <astronom><xsl:value-of select="@origin"/></astronom>
</xsl:for-each>
```

Le problème de cette instruction est qu’elle va afficher, pour chaque constellation, le nom de son découvreur. Le même résultat aurait pu être obtenu avec l’expression XPath `//constellation`. Pour dédoublonner les entrées, l’astuce réside dans la comparaison des identifiants uniques de chaque constellation :

```xml
<xsl:for-each select="//constellation[generate-id() = generate-id(key('astronoms', @origin)[1])]">
    <!--
        <astronom>Lacaille</astronom>
        <astronom>Ptolémée</astronom>
    -->
    <astronom><xsl:value-of select="@origin"/></astronom>
</xsl:for-each>
```

Dans le même ordre d’idée, le même résultat est obtenu en vérifiant que le nœud courant et le 1er élément des nœuds sélectionnés avec `key()` ne font qu’un :

```xml
<xsl:for-each select="//constellation[count( . | key('astronoms', @origin)[1]) = 1]">
    <astronom><xsl:value-of select="@origin"/></astronom>
</xsl:for-each>
```

### Le filtrage systématique

Bien plus facile à appréhender mais gourmande en ressources, cette technique exploite le lignage `preceding` pour sélectionner tous les nœuds qui précèdent (à l’exception des ancêtres, des attributs et des espaces de nommage).

Le nom du découvreur n’est retenu que s’il ne figure pas déjà dans ceux retenus précédemment :

```xml
<xsl:for-each select="//constellation[not(preceding::constellation/@origin = @origin)]">
    <astronom><xsl:value-of select="@origin"/></astronom>
</xsl:for-each>
```